In [1]:
# Imports

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm


# Regression Task in PyTorch

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing()
x_train, x_test, y_train, y_test = train_test_split(housing.data, housing.target)

In [4]:
type(x_train)

numpy.ndarray

In [5]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.reshape(-1,1).astype(np.float32))
y_test = torch.from_numpy(y_test.reshape(-1,1).astype(np.float32))

In [7]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(torch.Size([15480, 8]),
 torch.Size([15480, 1]),
 torch.Size([5160, 8]),
 torch.Size([5160, 1]))

In [8]:
# y_test.squeeze().shape

In [9]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

In [10]:
batch_size = 32 # The default in Keras
train_loader = torch.utils.data.DataLoader(
                        dataset=train_dataset,
                        batch_size=batch_size,
                        shuffle=False
                        )


test_loader = torch.utils.data.DataLoader(
                        dataset=test_dataset,
                        batch_size=batch_size,
                        shuffle=False # Not necessary!
                        )

In [11]:
class Model_Reg(nn.Module):
    def __init__(self, I):
        super(Model_Reg, self).__init__()
        self.linear1 = nn.Linear(I, 30)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(30, 1)
            
    def forward(self, X):
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        return out

In [12]:
modelr = Model_Reg(x_train.shape[1:][0])
from torchsummary import summary
summary(modelr, (1,x_train.shape[1:][0]))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 30]             270
              ReLU-2                [-1, 1, 30]               0
            Linear-3                 [-1, 1, 1]              31
Total params: 301
Trainable params: 301
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [13]:
# If GPU...
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
modelr.to(device)

cpu


Model_Reg(
  (linear1): Linear(in_features=8, out_features=30, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=30, out_features=1, bias=True)
)

In [14]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(modelr.parameters(), lr=0.001)

In [15]:
class PyTorchTrainer_Reg(object):
    def __init__(self,
                model,
                optim,
                loss_fn,
                device="cpu"):
        self.model = model
        self.optim = optim
        self.loss_fn = loss_fn
        self.device = device
        
    def fit(self,
            train_loader,
            test_loader,
            n_epochs: int=100,
            eval_inter: int=1):
        history = {}
        history['epoch'] = []
        history['training_loss'] = []
        history['test_loss'] = []
        
        # Training Loop

        history['p_test'] = []
        history['x_test'] = test_loader.dataset[:][0]
        history['y_test'] = test_loader.dataset[:][1]
        p_test = np.array([]) # for listing all predictions - last epoch only

        for epoch in range(n_epochs):
            history['epoch'].append(epoch+1)
            self.model.train()
            train_loss = []
            
            for inputs, targets in tqdm(train_loader, leave=False):
                # Move data to GPU
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                # zero the gradient
                self.optim.zero_grad()

                # forward pass
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, targets)

                # backward pass and optimize
                loss.backward()
                self.optim.step()

                train_loss.append(loss.item())

            train_loss = np.mean(train_loss)
            
            # save losses
            history['training_loss'].append(train_loss) 
    
            test_loss = []
            
            self.model.eval()
            with torch.no_grad():
                for inputs, targets in test_loader:
                    # Move data to GPU
                    inputs, targets = inputs.to(self.device), targets.to(self.device)

                    # forward pass
                    outputs_test = self.model(inputs)
                    loss_test = self.loss_fn(outputs_test, targets)

                    # update list of predictions  - done for last epoch only!
                    if epoch == (n_epochs - 1):
                        history['p_test'] = outputs_test.numpy()


                    test_loss.append(loss_test.item())
            history['p_test'] = p_test
            test_loss = np.mean(test_loss)
            # save losses
            history['test_loss'].append(test_loss) 
            
            if (epoch +1) % eval_inter == 0:
                print(f"Epoch: {epoch+1}/{n_epochs}, Train Loss: {history['training_loss'][-1]:.4f}, Test Loss: {history['test_loss'][-1]:.4f}", end='')
        return history
                
            

In [16]:
trainer = PyTorchTrainer_Reg(modelr, optimizer, loss_fn,device)

In [17]:
history={}
history = trainer.fit(train_loader, test_loader,20,1)

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 1/20, Train Loss: 1.9475, Test Loss: 0.9817

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 2/20, Train Loss: 0.7646, Test Loss: 0.7147

  0%|          | 0/484 [00:00<?, ?it/s]

...

Epoch: 18/20, Train Loss: 0.4576, Test Loss: 0.4905

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 19/20, Train Loss: 0.4550, Test Loss: 0.4877

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 20/20, Train Loss: 0.4526, Test Loss: 0.4851

# Skip Connections

In [18]:
class Model_Reg_SC(nn.Module):
    def __init__(self, I):
        super(Model_Reg_SC, self).__init__()
        self.linear1 = nn.Linear(I, 30)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(30, 30)
        self.linear3 = nn.Linear(38,1)
            
    def forward(self, X):
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = torch.cat((out,X), dim=-1)
        out = self.linear3(out)
        return out

In [19]:
modelr_sc = Model_Reg_SC(x_train.shape[1:][0])
from torchsummary import summary
summary(modelr_sc, (1,x_train.shape[1:][0]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 30]             270
              ReLU-2                [-1, 1, 30]               0
            Linear-3                [-1, 1, 30]             930
              ReLU-4                [-1, 1, 30]               0
            Linear-5                 [-1, 1, 1]              39
Total params: 1,239
Trainable params: 1,239
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [20]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(modelr_sc.parameters(), lr=0.001)

In [21]:
trainer = PyTorchTrainer_Reg(modelr_sc, optimizer, loss_fn,device)
history={}
history = trainer.fit(train_loader, test_loader,20,1)

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 1/20, Train Loss: 2.0801, Test Loss: 0.8087

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 2/20, Train Loss: 0.6777, Test Loss: 0.6668

  0%|          | 0/484 [00:00<?, ?it/s]

...

Epoch: 18/20, Train Loss: 0.4867, Test Loss: 0.5156

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 19/20, Train Loss: 0.4835, Test Loss: 0.5122

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 20/20, Train Loss: 0.4804, Test Loss: 0.5088

# Multiple Inputs and Skip Connections

In [22]:
x_train_A, x_train_B = x_train[:, :5], x_train[:, 2:]
x_test_A, x_test_B = x_test[:, :5], x_test[:, 2:]


train_dataset = torch.utils.data.TensorDataset(x_train_A, x_train_B, y_train)
test_dataset = torch.utils.data.TensorDataset(x_test_A, x_test_B, y_test)

In [23]:
batch_size = 32 # The default in Keras
train_loader = torch.utils.data.DataLoader(
                        dataset=train_dataset,
                        batch_size=batch_size,
                        shuffle=False
                        )


test_loader = torch.utils.data.DataLoader(
                        dataset=test_dataset,
                        batch_size=batch_size,
                        shuffle=False # Not necessary!
                        )

In [24]:
class Model_Reg_SCMI(nn.Module):
    def __init__(self, Ia, Ib):
        super(Model_Reg_SCMI, self).__init__()
        self.linear1b = nn.Linear(Ib, 30)
        self.relu = nn.ReLU()
        self.linear2b = nn.Linear(30, 30)
        self.linear3 = nn.Linear(35,1)
            
    def forward(self, Xa, Xb):
        out = self.linear1b(Xb)
        out = self.relu(out)
        out = self.linear2b(out)
        out = self.relu(out)
        out = torch.cat((Xa,out), dim=-1)
        out = self.linear3(out)
        return out

In [25]:
modelr_scmi = Model_Reg_SCMI(5,6)

from torchsummary import summary
summary(modelr_scmi, [(1,5), (1,6)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 30]             210
              ReLU-2                [-1, 1, 30]               0
            Linear-3                [-1, 1, 30]             930
              ReLU-4                [-1, 1, 30]               0
            Linear-5                 [-1, 1, 1]              36
Total params: 1,176
Trainable params: 1,176
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [26]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(modelr_scmi.parameters(), lr=0.001)

In [27]:
class PyTorchTrainer_RegMI(object):
    def __init__(self,
                model,
                optim,
                loss_fn,
                device="cpu"):
        self.model = model
        self.optim = optim
        self.loss_fn = loss_fn
        self.device = device
        
    def fit(self,
            train_loader,
            test_loader,
            n_epochs: int=100,
            eval_inter: int=1):
        history = {}
        history['epoch'] = []
        history['training_loss'] = []
        history['test_loss'] = []
        
        # Training Loop

        history['p_test'] = []
        history['x_test'] = test_loader.dataset[:][0]
        history['y_test'] = test_loader.dataset[:][1]
        p_test = np.array([]) # for listing all predictions - last epoch only

        for epoch in range(n_epochs):
            history['epoch'].append(epoch+1)
            self.model.train()
            train_loss = []
            
            for inputa, inputb, targets in tqdm(train_loader, leave=False):
                # Move data to GPU
                inputa, inputb, targets = inputa.to(self.device), inputb.to(self.device), targets.to(self.device)

                # zero the gradient
                self.optim.zero_grad()

                # forward pass
                outputs = self.model(inputa, inputb)
                loss = self.loss_fn(outputs, targets)

                # backward pass and optimize
                loss.backward()
                self.optim.step()

                train_loss.append(loss.item())

            train_loss = np.mean(train_loss)
            
            # save losses
            history['training_loss'].append(train_loss) 
    
            test_loss = []
            self.model.eval()
            with torch.no_grad():
                for inputa, inputb, targets in test_loader:
                    # Move data to GPU
                    inputa, inputb, targets = inputa.to(self.device), inputb.to(self.device), targets.to(self.device)

                    # forward pass
                    outputs_test = self.model(inputa, inputb)
                    loss_test = self.loss_fn(outputs_test, targets)

                    # update list of predictions  - done for last epoch only!
                    if epoch == (n_epochs - 1):
                        history['p_test'] = outputs_test.detach().numpy()


                    test_loss.append(loss_test.item())
            history['p_test'] = p_test
            test_loss = np.mean(test_loss)
            # save losses
            history['test_loss'].append(test_loss) 
            
            if (epoch +1) % eval_inter == 0:
                print(f"Epoch: {epoch+1}/{n_epochs}, Train Loss: {history['training_loss'][-1]:.4f}, Test Loss: {history['test_loss'][-1]:.4f}", end='')
        return history
                
            

In [28]:
trainer = PyTorchTrainer_RegMI(modelr_scmi, optimizer, loss_fn,device)
history={}
history = trainer.fit(train_loader, test_loader,20,1)

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 1/20, Train Loss: 2.1336, Test Loss: 0.9036

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 2/20, Train Loss: 0.7401, Test Loss: 0.6857

  0%|          | 0/484 [00:00<?, ?it/s]

...

Epoch: 18/20, Train Loss: 0.4799, Test Loss: 0.5063

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 19/20, Train Loss: 0.4767, Test Loss: 0.5031

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 20/20, Train Loss: 0.4737, Test Loss: 0.5002

# Multiple Outputs

In [29]:
class Model_Reg_SCMIO(nn.Module):
    def __init__(self, Ia, Ib):
        super(Model_Reg_SCMIO, self).__init__()
        self.linear1b = nn.Linear(Ib, 30)
        self.relu = nn.ReLU()
        self.linear2b = nn.Linear(30, 30)
        self.linear3 = nn.Linear(35,1)
        self.linear4 = nn.Linear(30,1)
            
    def forward(self, Xa, Xb):
        out = self.linear1b(Xb)
        out = self.relu(out)
        out = self.linear2b(out)
        outa = self.relu(out)
        out = torch.cat((Xa,outa), dim=-1)
        out = self.linear3(out)
        outb = self.linear4(outa)
        return out

In [30]:
modelr_scmio = Model_Reg_SCMIO(5,6)

from torchsummary import summary
summary(modelr_scmio, [(1,5), (1,6)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 30]             210
              ReLU-2                [-1, 1, 30]               0
            Linear-3                [-1, 1, 30]             930
              ReLU-4                [-1, 1, 30]               0
            Linear-5                 [-1, 1, 1]              36
            Linear-6                 [-1, 1, 1]              31
Total params: 1,207
Trainable params: 1,207
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [31]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(modelr_scmio.parameters(), lr=0.001)

In [32]:
trainer = PyTorchTrainer_RegMI(modelr_scmio, optimizer, loss_fn,device)
history={}
history = trainer.fit(train_loader, test_loader,20,1)

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 1/20, Train Loss: 2.0296, Test Loss: 0.8053

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 2/20, Train Loss: 0.6861, Test Loss: 0.6618

  0%|          | 0/484 [00:00<?, ?it/s]

...

Epoch: 18/20, Train Loss: 0.4793, Test Loss: 0.5085

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 19/20, Train Loss: 0.4750, Test Loss: 0.5039

  0%|          | 0/484 [00:00<?, ?it/s]

Epoch: 20/20, Train Loss: 0.4709, Test Loss: 0.4996

# Saving and Loading Models

In [33]:
torch.save(modelr_scmio.state_dict(), 'modelr_scmio.pth')

Here, recreate the model as above.

In [34]:
modelr_scmio.load_state_dict(torch.load('modelr_scmio.pth'))

<All keys matched successfully>

# Callbacks

Not applicable; all commands have to be entered in the PyTorch Trainer manually.

# Hyper-Parameter Tuning - Skorch

In [35]:
x_train, x_test, y_train, y_test = train_test_split(housing.data, housing.target)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train, x_test, y_train, y_test = x_train.astype(np.float32), x_test.astype(np.float32), y_train.reshape(-1,1).astype(np.float32), y_test.reshape(-1,1).astype(np.float32)

In [36]:
class Model_Reg_HP(nn.Module):
    def __init__(self, I=8, n_hidden=1, n_neurons=10):
        super(Model_Reg_HP, self).__init__()
        self.linear1 = nn.Linear(I, n_neurons)
        current_dim = n_neurons
        self.layers = nn.ModuleList()
        for _ in range(n_hidden):
            self.layers.append(nn.Linear(current_dim, n_neurons))
            self.layers.append(nn.ReLU())
            current_dim = n_neurons
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(n_neurons, 1)
            
    def forward(self, X, **kwargs):
        out = self.linear1(X)
        out = self.relu(out)
        for layer in self.layers[:-1]:
            out = layer(out)
        out = self.linear2(out)
        return out

In [37]:
modelr_hp = Model_Reg_HP(8,0,30)

from torchsummary import summary
summary(modelr_hp, (1,8))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 30]             270
              ReLU-2                [-1, 1, 30]               0
            Linear-3                 [-1, 1, 1]              31
Total params: 301
Trainable params: 301
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [38]:
params = {
    'module__n_hidden': [0, 1, 2, 3],
    'module__n_neurons': [10, 20, 30],
    'lr': [0.0005, 0.001, 0.002],
    'optimizer': [torch.optim.Adam, torch.optim.RMSprop]
}

In [39]:
from skorch import NeuralNetRegressor

net = NeuralNetRegressor(Model_Reg_HP, max_epochs=10, lr=0.001, verbose=0, train_split=False, iterator_train__shuffle=True)

## GridSearchCV

In [40]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(net, params, scoring='neg_mean_squared_error', refit=False, cv=2, verbose=2)

In [41]:
gs.fit(x_train, y_train)

Fitting 2 folds for each of 72 candidates, totalling 144 fits
[CV] END lr=0.0005, module__n_hidden=0, module__n_neurons=10, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.7s
[CV] END lr=0.0005, module__n_hidden=0, module__n_neurons=10, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.6s
[CV] END lr=0.0005, module__n_hidden=0, module__n_neurons=10, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.6s
[CV] END lr=0.0005, module__n_hidden=0, module__n_neurons=10, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.6s
[CV] END lr=0.0005, module__n_hidden=0, module__n_neurons=20, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.7s
[CV] END lr=0.0005, module__n_hidden=0, module__n_neurons=20, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.5s
[CV] END lr=0.0005, module__n_hidden=0, module__n_neurons=20, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.3s
[CV] END lr=0.0005, module__n_hidden=0, module__n_ne

[CV] END lr=0.001, module__n_hidden=1, module__n_neurons=20, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.5s
[CV] END lr=0.001, module__n_hidden=1, module__n_neurons=20, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.5s
[CV] END lr=0.001, module__n_hidden=1, module__n_neurons=20, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.6s
[CV] END lr=0.001, module__n_hidden=1, module__n_neurons=30, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.9s
[CV] END lr=0.001, module__n_hidden=1, module__n_neurons=30, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.5s
[CV] END lr=0.001, module__n_hidden=1, module__n_neurons=30, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.4s
[CV] END lr=0.001, module__n_hidden=1, module__n_neurons=30, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.8s
[CV] END lr=0.001, module__n_hidden=2, module__n_neurons=10, optimizer=<class 'torch.optim.adam.Adam'>; total time=

[CV] END lr=0.002, module__n_hidden=2, module__n_neurons=30, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.7s
[CV] END lr=0.002, module__n_hidden=3, module__n_neurons=10, optimizer=<class 'torch.optim.adam.Adam'>; total time=   2.0s
[CV] END lr=0.002, module__n_hidden=3, module__n_neurons=10, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.9s
[CV] END lr=0.002, module__n_hidden=3, module__n_neurons=10, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.9s
[CV] END lr=0.002, module__n_hidden=3, module__n_neurons=10, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.7s
[CV] END lr=0.002, module__n_hidden=3, module__n_neurons=20, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.8s
[CV] END lr=0.002, module__n_hidden=3, module__n_neurons=20, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.8s
[CV] END lr=0.002, module__n_hidden=3, module__n_neurons=20, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=

GridSearchCV(cv=2,
             estimator=<class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=<class '__main__.Model_Reg_HP'>,
),
             param_grid={'lr': [0.0005, 0.001, 0.002],
                         'module__n_hidden': [0, 1, 2, 3],
                         'module__n_neurons': [10, 20, 30],
                         'optimizer': [<class 'torch.optim.adam.Adam'>,
                                       <class 'torch.optim.rmsprop.RMSprop'>]},
             refit=False, scoring='neg_mean_squared_error', verbose=2)

In [42]:
print(gs.best_score_, gs.best_params_)

-0.3488439470529556 {'lr': 0.002, 'module__n_hidden': 3, 'module__n_neurons': 30, 'optimizer': <class 'torch.optim.rmsprop.RMSprop'>}


## RandomSearchCV

In [43]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(net, params, n_iter=20, scoring='neg_mean_squared_error', refit=False, cv=2, verbose=2)
rs.fit(x_train, y_train)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
[CV] END lr=0.002, module__n_hidden=2, module__n_neurons=10, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.6s
[CV] END lr=0.002, module__n_hidden=2, module__n_neurons=10, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.9s
[CV] END lr=0.001, module__n_hidden=0, module__n_neurons=20, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.6s
[CV] END lr=0.001, module__n_hidden=0, module__n_neurons=20, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.3s
[CV] END lr=0.002, module__n_hidden=0, module__n_neurons=10, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.3s
[CV] END lr=0.002, module__n_hidden=0, module__n_neurons=10, optimizer=<class 'torch.optim.adam.Adam'>; total time=   1.3s
[CV] END lr=0.002, module__n_hidden=0, module__n_neurons=20, optimizer=<class 'torch.optim.rmsprop.RMSprop'>; total time=   1.3s
[CV] END lr=0.002, module__n_hidden=0, module__n_neurons=20,

RandomizedSearchCV(cv=2,
                   estimator=<class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=<class '__main__.Model_Reg_HP'>,
),
                   n_iter=20,
                   param_distributions={'lr': [0.0005, 0.001, 0.002],
                                        'module__n_hidden': [0, 1, 2, 3],
                                        'module__n_neurons': [10, 20, 30],
                                        'optimizer': [<class 'torch.optim.adam.Adam'>,
                                                      <class 'torch.optim.rmsprop.RMSprop'>]},
                   refit=False, scoring='neg_mean_squared_error', verbose=2)

In [44]:
print(rs.best_score_, rs.best_params_)

-0.34611351788043976 {'optimizer': <class 'torch.optim.rmsprop.RMSprop'>, 'module__n_neurons': 30, 'module__n_hidden': 2, 'lr': 0.002}
